In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms

# Training settings
b_size = 512
n_epochs = 500
lr = 0.01
valid_step = 100
patience = max_patience = 2
best_loss = 100000
model_file = './models/model1.md'
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)    

def test_validate(model, device, test_loader, test_valid='Test'):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\n{} set: Average loss: {:.6f}, Accuracy: {}/{} ({:.6f}%)\n'.format(
        test_valid, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss
    
def train(model, device, train_loader, valid_loader, optimizer, epoch):
    global best_loss, patience
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
        if batch_idx % valid_step == 0:
            print('Start Validating...')
            valid_loss = test_validate(model, device, valid_loader, 'Valid')
            if valid_loss < best_loss:
                best_loss = valid_loss
                patience = max_patience
                print('Bese valid loss: {}'.format(best_loss))
                print('Improved! Reset patience.')
                print('Saving model...')
                torch.save(model, model_file)
            else:
                patience -= 1
                print('Not improved... Patience: {}'.format(patience))
                if patience == 0:
                    print('Out of patience. Stop training.')
                    return

def main():        
    dataroot = 'D:\Lab\dataset'

    device = torch.device("cuda")

    total_train_dataset = datasets.MNIST(dataroot, train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
    total_train_size = len(total_train_dataset)
    split_ratio = 0.8
    train_size = int(split_ratio * total_train_size)
    valid_size = total_train_size - train_size
    
    train_dataset, valid_dataset = random_split(total_train_dataset,[train_size, valid_size] )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=b_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=b_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(dataroot, train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=b_size, shuffle=True)


    model = Net().to(device)
    print(model)
    optimizer = optim.SGD(model.parameters(), lr=lr)

    for epoch in range(1, n_epochs + 1):
        train(model, device, train_loader, valid_loader, optimizer, epoch)
        if patience == 0:
            break
                
    print('Start testing...')
    model = torch.load(model_file)
    test_validate(model, device, test_loader)
        
main()

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)
Train Epoch: 1 [0/48000 (0%)]	Loss: 2.293011
Start Validating...

Valid set: Average loss: 2.300478, Accuracy: 1526/12000 (12.716667%)

Bese valid loss: 2.300478332519531
Improved! Reset patience.
Saving model...


C:\Users\Likit\Anaconda3\envs\tensorflow\lib\site-packages\torch\serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Train Epoch: 1 [25600/48000 (53%)]	Loss: 2.122329
Train Epoch: 2 [0/48000 (0%)]	Loss: 1.863460
Start Validating...

Valid set: Average loss: 1.835186, Accuracy: 7903/12000 (65.858333%)

Bese valid loss: 1.8351863174438476
Improved! Reset patience.
Saving model...
Train Epoch: 2 [25600/48000 (53%)]	Loss: 1.370900
Train Epoch: 3 [0/48000 (0%)]	Loss: 1.067566
Start Validating...

Valid set: Average loss: 1.009373, Accuracy: 9406/12000 (78.383333%)

Bese valid loss: 1.0093730061848958
Improved! Reset patience.
Saving model...
Train Epoch: 3 [25600/48000 (53%)]	Loss: 0.804502
Train Epoch: 4 [0/48000 (0%)]	Loss: 0.806520
Start Validating...

Valid set: Average loss: 0.664694, Accuracy: 10037/12000 (83.641667%)

Bese valid loss: 0.6646940180460612
Improved! Reset patience.
Saving model...
Train Epoch: 4 [25600/48000 (53%)]	Loss: 0.601481
Train Epoch: 5 [0/48000 (0%)]	Loss: 0.646774
Start Validating...

Valid set: Average loss: 0.530001, Accuracy: 10313/12000 (85.941667%)

Bese valid loss: 0.5

Train Epoch: 32 [25600/48000 (53%)]	Loss: 0.208314
Train Epoch: 33 [0/48000 (0%)]	Loss: 0.443385
Start Validating...

Valid set: Average loss: 0.228893, Accuracy: 11189/12000 (93.241667%)

Bese valid loss: 0.2288929697672526
Improved! Reset patience.
Saving model...
Train Epoch: 33 [25600/48000 (53%)]	Loss: 0.178523
Train Epoch: 34 [0/48000 (0%)]	Loss: 0.378243
Start Validating...

Valid set: Average loss: 0.225013, Accuracy: 11212/12000 (93.433333%)

Bese valid loss: 0.22501282755533855
Improved! Reset patience.
Saving model...
Train Epoch: 34 [25600/48000 (53%)]	Loss: 0.151585
Train Epoch: 35 [0/48000 (0%)]	Loss: 0.413321
Start Validating...

Valid set: Average loss: 0.222560, Accuracy: 11215/12000 (93.458333%)

Bese valid loss: 0.22255968125661213
Improved! Reset patience.
Saving model...
Train Epoch: 35 [25600/48000 (53%)]	Loss: 0.221841
Train Epoch: 36 [0/48000 (0%)]	Loss: 0.358542
Start Validating...

Valid set: Average loss: 0.218865, Accuracy: 11227/12000 (93.558333%)

Bese val

Train Epoch: 63 [25600/48000 (53%)]	Loss: 0.169301
Train Epoch: 64 [0/48000 (0%)]	Loss: 0.345619
Start Validating...

Valid set: Average loss: 0.161791, Accuracy: 11416/12000 (95.133333%)

Bese valid loss: 0.1617906405131022
Improved! Reset patience.
Saving model...
Train Epoch: 64 [25600/48000 (53%)]	Loss: 0.130878
Train Epoch: 65 [0/48000 (0%)]	Loss: 0.362748
Start Validating...

Valid set: Average loss: 0.159994, Accuracy: 11420/12000 (95.166667%)

Bese valid loss: 0.1599937030474345
Improved! Reset patience.
Saving model...
Train Epoch: 65 [25600/48000 (53%)]	Loss: 0.109281
Train Epoch: 66 [0/48000 (0%)]	Loss: 0.252857
Start Validating...

Valid set: Average loss: 0.158450, Accuracy: 11429/12000 (95.241667%)

Bese valid loss: 0.1584496482213338
Improved! Reset patience.
Saving model...
Train Epoch: 66 [25600/48000 (53%)]	Loss: 0.122963
Train Epoch: 67 [0/48000 (0%)]	Loss: 0.307200
Start Validating...

Valid set: Average loss: 0.157932, Accuracy: 11429/12000 (95.241667%)

Bese valid

Train Epoch: 94 [25600/48000 (53%)]	Loss: 0.089745
Train Epoch: 95 [0/48000 (0%)]	Loss: 0.278159
Start Validating...

Valid set: Average loss: 0.129328, Accuracy: 11526/12000 (96.050000%)

Bese valid loss: 0.129327561378479
Improved! Reset patience.
Saving model...
Train Epoch: 95 [25600/48000 (53%)]	Loss: 0.092602
Train Epoch: 96 [0/48000 (0%)]	Loss: 0.269024
Start Validating...

Valid set: Average loss: 0.128329, Accuracy: 11533/12000 (96.108333%)

Bese valid loss: 0.1283290220896403
Improved! Reset patience.
Saving model...
Train Epoch: 96 [25600/48000 (53%)]	Loss: 0.110500
Train Epoch: 97 [0/48000 (0%)]	Loss: 0.255045
Start Validating...

Valid set: Average loss: 0.127976, Accuracy: 11526/12000 (96.050000%)

Bese valid loss: 0.12797633504867553
Improved! Reset patience.
Saving model...
Train Epoch: 97 [25600/48000 (53%)]	Loss: 0.151630
Train Epoch: 98 [0/48000 (0%)]	Loss: 0.263459
Start Validating...

Valid set: Average loss: 0.126733, Accuracy: 11547/12000 (96.225000%)

Bese valid


Valid set: Average loss: 0.112449, Accuracy: 11583/12000 (96.525000%)

Not improved... Patience: 1
Train Epoch: 125 [25600/48000 (53%)]	Loss: 0.057659
Train Epoch: 126 [0/48000 (0%)]	Loss: 0.262655
Start Validating...

Valid set: Average loss: 0.111024, Accuracy: 11590/12000 (96.583333%)

Bese valid loss: 0.1110244034131368
Improved! Reset patience.
Saving model...
Train Epoch: 126 [25600/48000 (53%)]	Loss: 0.073446
Train Epoch: 127 [0/48000 (0%)]	Loss: 0.241247
Start Validating...

Valid set: Average loss: 0.111038, Accuracy: 11599/12000 (96.658333%)

Not improved... Patience: 1
Train Epoch: 127 [25600/48000 (53%)]	Loss: 0.064763
Train Epoch: 128 [0/48000 (0%)]	Loss: 0.221885
Start Validating...

Valid set: Average loss: 0.110640, Accuracy: 11596/12000 (96.633333%)

Bese valid loss: 0.11064029598236085
Improved! Reset patience.
Saving model...
Train Epoch: 128 [25600/48000 (53%)]	Loss: 0.088128
Train Epoch: 129 [0/48000 (0%)]	Loss: 0.185259
Start Validating...

Valid set: Average los


Valid set: Average loss: 0.099689, Accuracy: 11638/12000 (96.983333%)

Bese valid loss: 0.09968889784812927
Improved! Reset patience.
Saving model...
Train Epoch: 157 [25600/48000 (53%)]	Loss: 0.081611
Train Epoch: 158 [0/48000 (0%)]	Loss: 0.208468
Start Validating...

Valid set: Average loss: 0.099667, Accuracy: 11630/12000 (96.916667%)

Bese valid loss: 0.0996665693918864
Improved! Reset patience.
Saving model...
Train Epoch: 158 [25600/48000 (53%)]	Loss: 0.057177
Train Epoch: 159 [0/48000 (0%)]	Loss: 0.251386
Start Validating...

Valid set: Average loss: 0.099418, Accuracy: 11638/12000 (96.983333%)

Bese valid loss: 0.09941817569732667
Improved! Reset patience.
Saving model...
Train Epoch: 159 [25600/48000 (53%)]	Loss: 0.100181
Train Epoch: 160 [0/48000 (0%)]	Loss: 0.266887
Start Validating...

Valid set: Average loss: 0.099174, Accuracy: 11639/12000 (96.991667%)

Bese valid loss: 0.09917388693491618
Improved! Reset patience.
Saving model...
Train Epoch: 160 [25600/48000 (53%)]	Los